## 전자상거래 배송 데이터

제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기

학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 

이를 평가용 데이터(X_test)에 적용하여 얻은 예측 확률값을 

다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [4]:
x_train.nunique()

ID                     6598
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Product_importance        3
Gender                    2
Discount_offered         65
Weight_in_gms          3365
dtype: int64

In [7]:
x_test

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6811,D,Ship,5,2,259,5,low,F,7,1032
1,4320,F,Ship,3,5,133,3,medium,F,4,5902
2,5732,F,Road,3,4,191,5,medium,F,4,4243
3,7429,D,Ship,4,2,221,3,low,M,10,4126
4,2191,D,Flight,4,5,230,2,low,F,38,2890
...,...,...,...,...,...,...,...,...,...,...,...
4396,2610,F,Flight,4,1,157,3,medium,M,31,1712
4397,3406,B,Road,3,5,139,2,medium,M,7,5536
4398,10395,A,Road,4,1,170,10,medium,F,3,5211
4399,3646,B,Ship,3,1,244,3,medium,F,1,5695


In [8]:
drop_col = ['ID']
x_train_drop = x_train.drop(columns = drop_col)
x_test_drop = x_test.drop(columns = drop_col)

In [9]:
x_train_dum = pd.get_dummies(x_train_drop)
x_test_dum = pd.get_dummies(x_test_drop)

In [13]:
x_test_dum = x_test_dum[x_train_dum.columns]

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score

In [16]:
y = y_train['Reached.on.Time_Y.N']

In [17]:
x_train, x_val, y_train, y_val = train_test_split(x_train_dum, y, stratify=y, random_state = 42)
rf = RandomForestClassifier(random_state = 23)
rf.fit(x_train, y_train)

RandomForestClassifier(random_state=23)

In [22]:
pred = rf.predict_proba(x_val)[:,1]
roc_auc_score(y_val, pred)

0.7432181977787107

In [ ]:
# #model_score
# predict_train_label = rf.predict(X_train)
# predict_train_proba = rf.predict_proba(X_train)[:,1]

# predict_validation_label = rf.predict(X_validation)
# predict_validation_prob = rf.predict_proba(X_validation)[:,1]

# print('validation auc :', roc_auc_score(Y_validation,predict_validation_prob))

In [19]:
test_pred = rf.predict_proba(x_test_dum)[:,1]

In [20]:
pd.DataFrame({'pred': test_pred}).to_csv('007005513.csv', index=False)

In [21]:
test = pd.read_csv('007005513.csv')
test

,pred
0,0.58
1,0.46
2,0.54
3,0.58
4,1.00
...,...
4396,1.00
4397,0.56
4398,0.30
4399,0.31
